In [21]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords #common words to filter using data
from sklearn.metrics.pairwise import linear_kernel #can be applied to calculate the similarity between different travel destinations or places based on their attributes or features

from sklearn.feature_extraction.text import CountVectorizer #converts collection of documents into matrix of words, coz ML data basically reprsented in amtrix form.
from sklearn.feature_extraction.text import TfidfVectorizer #used to analyze and extract important keywords and phrases from this text. 
from sklearn.decomposition import LatentDirichletAllocation #extract specific places like beach ,mountain
from sklearn.metrics.pairwise import cosine_similarity #calculate the similarity between two destinations based on user ratings, location, or other attributes 
from sklearn.feature_extraction.text import CountVectorizer #word to matrix
import re #regular expressions module
import random #suggest random destinations at home page
import cufflinks # creates user friendly charts and reports if we wanna track
pd.options.display.max_columns = 30 # tell panda to show max 30 coloumns when printing data
from IPython.core.interactiveshell import InteractiveShell # interactive features for working with python code
import plotly.figure_factory as ff #interactive plots and charts
cufflinks.go_offline() #can generate plots and visualizations without needing to go online
cufflinks.set_config_file(world_readable=True, theme='solar') #allows generated plots to be read by others, solar is like color  scehemes

In [51]:
df = pd.read_csv('Travel_Data.csv',encoding="latin-1") #uses pandas library for reading the file
df.head()# quick overview of data
print('We have', len(df), 'Travel places in the data') #total places in our dataframe

We have 225 Travel places in the data


In [52]:
df.head()

,ID,Name,Type,Description
0,1,"Luksentyre, Outer Hebrides, Scotland",Beach,At the end of a winding road on the wind-batte...
1,2,"Dune Du Pilat, France",Beach,"In the warmer months, Europeï¿½s tallest sand ..."
2,3,Ora Beach Indonesia,Beach,"With a higgledy-piggledy 37,000-mile coastline..."
3,4,Awaroa South Island New Zealand,Beach,Set deep in the heart of New Zealandï¿½s glori...
4,5,Die Plast Walker Bay Nature Reserve South Africa,Beach,Donï¿½t be surprised to spot the occasional so...


In [53]:
ex = df[df.index==0].values[0] #values is aNumPy array, all of same data type string.
ex  #extracts first row of data and stores it in ex

array([1, 'Luksentyre, Outer Hebrides, Scotland', 'Beach',
       'At the end of a winding road on the wind-battered west coast of the Isle of Harris, youï¿½ll find something to make your eyes pop. Luskentyre Beach is a paint splash of brilliant white, surrounded by almost impossibly azure water. On a sunny day, itï¿½s so unerringly blue you may find it hard to believe you are at a beach in Scotland at all. But regardless of its paradoxically tropical feel, itï¿½s at its most beautiful when eel-coloured storm clouds roll in and seas turn flinty, turning this dramatic landscape an eerie monochrome.'],
      dtype=object)

In [54]:
import nltk
nltk.download('stopwords') #common word like 'the', 'and', 'in that are filtered out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shwen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]') #identify these chars and replace them with space to clean the data
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') #remove characters which arent in the mentioned set
STOPWORDS = set(stopwords.words('english')) #stopwords.words('english')) retrievs common english stopwords & filter them out

def clean_text(text): #functionn which takes string argument ,string name=text
    """
        text: a string  #provides purpose of function
        
        return: modified initial string
    """
    text = text.lower() # lower is inbuilt function which returns all lowercase words
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # split will sepratae words wheever spaces, then word for word iterates over
    return text                                                             #and only includes the word if its not in stopwords set
    
df['Description_Clean'] = df['Description'].apply(clean_text)
#applies cleanText to description coloumn,after cleaning assigns it to new "Description_clean" coloumn.

<>:1: SyntaxWarning:

invalid escape sequence '\['

<>:1: SyntaxWarning:

invalid escape sequence '\['

C:\Users\shwen\AppData\Local\Temp\ipykernel_12548\3055159535.py:1: SyntaxWarning:

invalid escape sequence '\['



In [57]:
df.columns = df.columns.str.strip()
df.set_index('Name',inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.0, stop_words='english') 
tfidf_matrix = tf.fit_transform(df['Description_Clean'])    #transforms DC coloumn into dc matrix   # min_df will include all words irregardless of frequency
indices = pd.Series(df.index) #indices variables creates index od all dataframes, convinient to look later

def get_recommendations(user_arr): #function, has paramter user_arr likely to generate travel reccommendation
  rec_places = [] #list for travel recommendation
  vectors = [] #used for storing tfid vectors
  idx = [] #indices of rec_places list
  for u in user_arr: #iterates through all user_arr elements
    idx.append(indices[indices == u].index[0]) #indices=u checks which element from indices series(early)= u in user_array, in[whole] creates series of previous condition ka equal elements
  for i in idx:   #iterates through idx list                             #stores in the idx list
    vectors.append(tfidf_matrix[i]) #This code retrieves the TF-IDF vector associated with the user data at index i 
                                    #in the tfidf_matrix and appends it to the vectors list.

  avg_vec = sum(vectors)/len(vectors) #mean of vectors
  avg_vec_sim = linear_kernel(avg_vec, tfidf_matrix)  #cosine similarity between average vector & each vector in matrix, result in sim variable

  avg_vec_sim = avg_vec_sim.flatten() #1-d array

  top_10_rec = pd.Series(avg_vec_sim).sort_values(ascending = False) #converts avg_vec_sim into pandas series, and sorts in descending order, highest similarity first
  top_10_indexes = list(top_10_rec.iloc[1:len(user_arr)+11].index) #takes indexes from index 1 to index 10,list of indices is indexes of top10 locations
  rec_places = []
  for i in top_10_indexes: #iterate through top10 indexes
        rec_places.append([list(df.index)[i], df['Type'][i]]) #in rec_places list append df.index=name of destination, type of destination

  return rec_places #return recommendations to user



KeyError: "None of ['Name'] are in the columns"

In [61]:
user_arr = ['Grand Canyon Rim-to-Rim, Arizona'] 
rec_places = get_recommendations(user_arr)

C:\Users\shwen\AppData\Local\Temp\ipykernel_12548\3290784008.py:26: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [62]:
rec_places = [i for i in rec_places if i[0] not in user_arr]

In [63]:
rec_places

[['Buckskin Gulch to Paria Canyon via Wire Pass Trailhead', 'Hiking/Mountain'],
 ['Great Wall, China', 'Historical Monument'],
 ['Teton Crest Trail Grand Teton National Park, Wyoming', 'Hiking/Mountain'],
 ['Greenstone Ridge Trail, Michigan', 'Hiking/Mountain'],
 ['Tour du Mont Blanc Switzerland, Italy, France', 'Hiking/Mountain'],
 ['Rainforests of Brunei', 'Adventure/Forest'],
 ['Wonderland Trail Mount Rainier National Park, Washington',
  'Hiking/Mountain'],
 ['Kalalau Trail Kauai, Hawaii', 'Hiking/Mountain'],
 ['Kepler Track Fiordland National Park, New Zealand', 'Hiking/Mountain'],
 ['Superior Hiking Trail Duluth, Minnesota, to the Canadian Border',
  'Hiking/Mountain'],
 ['Panorama Ridge Garibaldi Provincial Park, Canada', 'Hiking/Mountain']]